<a href="https://colab.research.google.com/github/Benylaode/model_tfidf/blob/main/Model_Rekomendasi_Buku_(Python).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [15]:

try:
    df = pd.read_csv('Data_Books.csv')
    print("Dataset berhasil dimuat.")
except FileNotFoundError:
    print("File 'Data_Books.csv' tidak ditemukan. Menggunakan data dummy untuk demonstrasi.")
    data = {
        'title': [
            'The Lord of the Rings', 'The Hobbit', 'Pride and Prejudice',
            'Sense and Sensibility', 'To Kill a Mockingbird', '1984',
            'Animal Farm', 'The Great Gatsby', 'Moby Dick', 'War and Peace'
        ],
        'authors': [
            'J.R.R. Tolkien', 'J.R.R. Tolkien', 'Jane Austen',
            'Jane Austen', 'Harper Lee', 'George Orwell',
            'George Orwell', 'F. Scott Fitzgerald', 'Herman Melville', 'Leo Tolstoy'
        ],
        'categories': [
            'Fantasy', 'Fantasy', 'Romance',
            'Romance', 'Classics', 'Dystopian',
            'Satire', 'Classics', 'Adventure', 'Historical Fiction'
        ]
    }
    df = pd.DataFrame(data)


File 'Data_Books.csv' tidak ditemukan. Menggunakan data dummy untuk demonstrasi.


In [16]:
df.fillna('', inplace=True)

df['combined_features'] = df['title'] + ' ' + df['authors'] + ' ' + df['categories']
print("\nFitur teks berhasil digabungkan.")
print(df[['title', 'authors', 'categories', 'combined_features']].head())



Fitur teks berhasil digabungkan.
                   title         authors categories  \
0  The Lord of the Rings  J.R.R. Tolkien    Fantasy   
1             The Hobbit  J.R.R. Tolkien    Fantasy   
2    Pride and Prejudice     Jane Austen    Romance   
3  Sense and Sensibility     Jane Austen    Romance   
4  To Kill a Mockingbird      Harper Lee   Classics   

                              combined_features  
0  The Lord of the Rings J.R.R. Tolkien Fantasy  
1             The Hobbit J.R.R. Tolkien Fantasy  
2       Pride and Prejudice Jane Austen Romance  
3     Sense and Sensibility Jane Austen Romance  
4     To Kill a Mockingbird Harper Lee Classics  


In [17]:

tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)

tfidf_matrix = tfidf_vectorizer.fit_transform(df['combined_features'])
print(f"\nUkuran matriks TF-IDF: {tfidf_matrix.shape}")
print("TF-IDF Vectorizer berhasil diterapkan.")


Ukuran matriks TF-IDF: (10, 39)
TF-IDF Vectorizer berhasil diterapkan.


In [18]:

cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
print(f"\nUkuran matriks Cosine Similarity: {cosine_sim.shape}")
print("Cosine Similarity berhasil dihitung.")


Ukuran matriks Cosine Similarity: (10, 10)
Cosine Similarity berhasil dihitung.


In [19]:
class BookRecommender:
    def __init__(self, dataframe, cosine_sim_matrix, indices):
        self.df = dataframe
        self.cosine_sim = cosine_sim_matrix
        self.indices = indices

    def recommend_books(self, title, num_recommendations=5):
        if title not in self.indices:
            print(f"Buku dengan judul '{title}' tidak ditemukan dalam dataset.")
            return []

        idx = self.indices[title]

        sim_scores = list(enumerate(self.cosine_sim[idx]))

        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

        sim_scores = sim_scores[1:num_recommendations+1]

        book_indices = [i[0] for i in sim_scores]

        recommended_titles = self.df['title'].iloc[book_indices].tolist()
        return recommended_titles



In [20]:
indices = pd.Series(df.index, index=df['title']).drop_duplicates()

recommender = BookRecommender(df, cosine_sim, indices)
print("\nObjek BookRecommender berhasil diinisialisasi.")

book_title_to_recommend = 'The Hobbit'
print(f"\nMencari rekomendasi untuk buku: '{book_title_to_recommend}'")
recommendations = recommender.recommend_books(book_title_to_recommend, num_recommendations=5)

if recommendations:
    print(f"Rekomendasi buku untuk '{book_title_to_recommend}':")
    for i, book in enumerate(recommendations):
        print(f"{i+1}. {book}")
else:
    print("Tidak ada rekomendasi yang ditemukan.")

book_title_to_recommend_not_found = 'Buku Aneh'
print(f"\nMencari rekomendasi untuk buku: '{book_title_to_recommend_not_found}'")
recommendations_not_found = recommender.recommend_books(book_title_to_recommend_not_found)


Objek BookRecommender berhasil diinisialisasi.

Mencari rekomendasi untuk buku: 'The Hobbit'
Rekomendasi buku untuk 'The Hobbit':
1. The Lord of the Rings
2. Pride and Prejudice
3. Sense and Sensibility
4. To Kill a Mockingbird
5. 1984

Mencari rekomendasi untuk buku: 'Buku Aneh'
Buku dengan judul 'Buku Aneh' tidak ditemukan dalam dataset.
